# QRT Portfolio Optimisation

In [177]:
import pandas as pd
import lseg.data as ld
from pprint import pprint

ld.open_session()

print("Connected to Workspace")

Connected to Workspace


## Fetch the Data
We import the universe that we wish to operate in as well as the various factors that we would like to use

In [178]:
# The universe of equities we will be dealing with
with open('universe.txt') as f:
    universe = f.read().splitlines()

print("Universe of size", len(universe))
print(universe)

field_names = []
fields = {} # fields[name] = {}

class field:
    name = ""
    lag = 0
    string_rep = ""
    
    def __repr__(self):
        return f"ric = {self.name}, lag = {self.lag} ({self.string_rep})"
    
# The various fields we will be using
with open('technical_fields.txt', 'r') as f:
    for line in f:
        field_name, lag = line.split()
        field_names.append(field_name)
        fields[field_name] = field()
        fields[field_name].name = field_name
        fields[field_name].lag = int(lag)

# with open('fundamental_fields.txt', 'r') as f:
#     for line in f:
#         name = line.strip()
#         fields[name] = 0

print("Fields: ", fields.keys())

Universe of size 25
['AAPL.O', 'AMZN.O', 'AVGO.O', 'BAC', 'BRKb', 'COST.O', 'CRM', 'GOOG.O', 'GOOGL.O', 'HD', 'JNJ', 'JPM', 'LLY', 'MA', 'META.O', 'MSFT.O', 'NFLX.O', 'NVDA.O', 'ORCL.K', 'PG', 'TSLA.O', 'UNH', 'V', 'WMT', 'XOM']
Fields:  dict_keys(['TR.TotalReturn1Wk', 'TR.Volume', 'TR.CompanyMarketCapitalization', 'TR.PriceClose', 'TR.PriceOpen'])


In [179]:
start_date = "2024-01-01"
end_date = "2024-10-01"

start_date = pd.to_datetime(start_date)
end_date = pd.to_datetime(end_date)

print("Start Date: ", start_date)
print("End Date: ", end_date)

start_date = start_date.isoformat() + 'Z'
end_date = end_date.isoformat() + 'Z'

Start Date:  2024-01-01 00:00:00
End Date:  2024-10-01 00:00:00


In [180]:
# Get a sample of the data for the universe of equities
data = ld.get_history(universe, list(fields.keys()), start=start_date, end=end_date, interval='1d')
print("Data shape: ", data.shape)
data.head()

Data shape:  (189, 125)


c:\Users\tangy\Desktop\PortfolioManagement\qrtVenv\lib\site-packages\pandas\core\dtypes\cast.py:1056:RuntimeWarning: invalid value encountered in cast
c:\Users\tangy\Desktop\PortfolioManagement\qrtVenv\lib\site-packages\pandas\core\dtypes\cast.py:1080:RuntimeWarning: invalid value encountered in cast


AAPL.O                                          \
           1 Week Total Return    Volume Company Market Capitalization   
Date                                                                     
2024-01-02           -3.838384  82488674                 2870346373440   
2024-01-03           -4.607818  58414460                 2848854338000   
2024-01-04           -6.028515  71983570                 2812673501360   
2024-01-05           -5.895185  62379661                 2801386317280   
2024-01-08           -3.620215  59144470                 2869109421760   

                                               AMZN.O            \
           Price Close Price Open 1 Week Total Return    Volume   
Date                                                              
2024-01-02      185.64     187.15           -2.268431  47339424   
2024-01-03      184.25     184.22           -3.175949  49425495   
2024-01-04      181.91     182.15           -5.743904  56039807   
2024-01-05      181.18     181.99           -4.409635  45153147   
2024-01-08      185.56    182.085           -1.869159  46757053   

                                                                 ...  \
           Company Market Capitalization Price Close Price Open  ...   
Date                                                             ...   
2024-01-02               1556723190000.0      149.93     151.54  ...   
2024-01-03               1541564010000.0      148.47      149.2  ...   
2024-01-04               1501070310000.0      144.57     145.59  ...   
2024-01-05               1508026920000.0      145.24     144.69  ...   
2024-01-08               1548105300000.0       149.1     146.74  ...   

                           WMT                                          \
           1 Week Total Return    Volume Company Market Capitalization   
Date                                                                     
2024-01-02            1.841314  23539700           428845481177.945007   
2024-01-03            0.899417  18756078           428872400822.741028   
2024-01-04            0.120581  19282489            424726363612.35498   
2024-01-05           -0.596258  21709524           421899518224.205017   
2024-01-08             0.38059  20680898           426045555434.591003   

                                                  XOM            \
           Price Close Price Open 1 Week Total Return    Volume   
Date                                                              
2024-01-02   53.096614  52.416614            0.215391  23483049   
2024-01-03   53.099947  53.256613            1.534527  23490779   
2024-01-04   52.586614  53.069947            2.125961  19395154   
2024-01-05   52.236614  52.639947             2.65053  15827392   
2024-01-08   52.749947  52.233281            0.940188  23370127   

                                                                 
           Company Market Capitalization Price Close Price Open  
Date                                                             
2024-01-02                406471560000.0      102.36     100.92  
2024-01-03                409886620000.0      103.22     102.27  
2024-01-04                406312720000.0      102.32     104.08  
2024-01-05                407543730000.0      102.63     103.17  
2024-01-08                400753320000.0      100.92     100.73  

[5 rows x 125 columns]

In [181]:
# Export the data to a CSV file
data.to_csv('data.csv')

In [182]:
for i in range(len(field_names)):
    fields[field_names[i]].string_rep = data.columns[i][1]

pprint(fields)

{'TR.CompanyMarketCapitalization': ric = TR.CompanyMarketCapitalization, lag = 1 (Company Market Capitalization),
 'TR.PriceClose': ric = TR.PriceClose, lag = 2 (Price Close),
 'TR.PriceOpen': ric = TR.PriceOpen, lag = 2 (Price Open),
 'TR.TotalReturn1Wk': ric = TR.TotalReturn1Wk, lag = 0 (1 Week Total Return),
 'TR.Volume': ric = TR.Volume, lag = 1 (Volume)}


## Construct the Dataset
Our data is based on time series. We would do well to append columns of lagged data to the data set to allow the model to refer to past data.
We also construct a target variable that represents the percentage change in the close price in the next $q$ days. The idea is to rebalance every $q$ days.

In [183]:
# Construct the lagged fields

def expand_data(data_in): 
    expanded_data = data_in.copy()
    for ric in universe:
        for field_name in field_names:
            lag = fields[field_name].lag
            string_rep = fields[field_name].string_rep
            for cur_lag in range(1, lag+1):
                expanded_data[(ric, f'{string_rep}_lag_{cur_lag}')] = data_in[(ric, f'{string_rep}')].shift(cur_lag)
    expanded_data = expanded_data.sort_index(axis=1)
    expanded_data.dropna(inplace=True)
    return expanded_data
       
# Sort the lagged fields back into the correct order
expanded_data = expand_data(data)
expanded_data.head()

AAPL.O                                \
           1 Week Total Return Company Market Capitalization   
Date                                                           
2024-01-04           -6.028515                 2812673501360   
2024-01-05           -5.895185                 2801386317280   
2024-01-08           -3.620215                 2869109421760   
2024-01-09           -0.269339                 2862615425440   
2024-01-10            1.052917                 2878850416240   

                                                                              \
           Company Market Capitalization_lag_1 Price Close Price Close_lag_1   
Date                                                                           
2024-01-04                       2848854338000      181.91            184.25   
2024-01-05                       2812673501360      181.18            181.91   
2024-01-08                       2801386317280      185.56            181.18   
2024-01-09                       2869109421760      185.14            185.56   
2024-01-10                       2862615425440      186.19            185.14   

                                                                           \
           Price Close_lag_2 Price Open Price Open_lag_1 Price Open_lag_2   
Date                                                                        
2024-01-04            185.64     182.15           184.22           187.15   
2024-01-05            184.25     181.99           182.15           184.22   
2024-01-08            181.91    182.085           181.99           182.15   
2024-01-09            181.18     183.92          182.085           181.99   
2024-01-10            185.56     184.35           183.92          182.085   

                      ...                           XOM  \
              Volume  ... Company Market Capitalization   
Date                  ...                                 
2024-01-04  71983570  ...                406312720000.0   
2024-01-05  62379661  ...                407543730000.0   
2024-01-08  59144470  ...                400753320000.0   
2024-01-09  42841809  ...                395789570000.0   
2024-01-10  46792908  ...                391897990000.0   

                                                                              \
           Company Market Capitalization_lag_1 Price Close Price Close_lag_1   
Date                                                                           
2024-01-04                      409886620000.0      102.32            103.22   
2024-01-05                      406312720000.0      102.63            102.32   
2024-01-08                      407543730000.0      100.92            102.63   
2024-01-09                      400753320000.0       99.67            100.92   
2024-01-10                      395789570000.0       98.69             99.67   

                                                                           \
           Price Close_lag_2 Price Open Price Open_lag_1 Price Open_lag_2   
Date                                                                        
2024-01-04            102.36     104.08           102.27           100.92   
2024-01-05            103.22     103.17           104.08           102.27   
2024-01-08            102.32     100.73           103.17           104.08   
2024-01-09            102.63     101.29           100.73           103.17   
2024-01-10            100.92       99.8           101.29           100.73   

                                   
              Volume Volume_lag_1  
Date                               
2024-01-04  19395154     23490779  
2024-01-05  15827392     19395154  
2024-01-08  23370127     15827392  
2024-01-09  19496647     23370127  
2024-01-10  18206088     19496647  

[5 rows x 275 columns]

In [184]:
expanded_data.to_csv('expanded_data.csv')

In [185]:
pred_horizon = 2 # we predict price close in 2 days

# Construct the target variable
for ric in universe:
    expanded_data[(ric, 'target')] = 100 * (data[(ric, 'Price Close')].shift(-pred_horizon) - data[(ric, 'Price Close')]) / data[(ric, 'Price Close')]
    
expanded_data = expanded_data.dropna()
targets = expanded_data[[(ric, 'target') for ric in universe]]


In [186]:
X = expanded_data.drop(columns=[(ric, 'target') for ric in universe])
y = targets
y.columns = y.columns.droplevel(1)

In [187]:
X.head()

AAPL.O                                \
           1 Week Total Return Company Market Capitalization   
Date                                                           
2024-01-04           -6.028515                 2812673501360   
2024-01-05           -5.895185                 2801386317280   
2024-01-08           -3.620215                 2869109421760   
2024-01-09           -0.269339                 2862615425440   
2024-01-10            1.052917                 2878850416240   

                                                                              \
           Company Market Capitalization_lag_1 Price Close Price Close_lag_1   
Date                                                                           
2024-01-04                       2848854338000      181.91            184.25   
2024-01-05                       2812673501360      181.18            181.91   
2024-01-08                       2801386317280      185.56            181.18   
2024-01-09                       2869109421760      185.14            185.56   
2024-01-10                       2862615425440      186.19            185.14   

                                                                           \
           Price Close_lag_2 Price Open Price Open_lag_1 Price Open_lag_2   
Date                                                                        
2024-01-04            185.64     182.15           184.22           187.15   
2024-01-05            184.25     181.99           182.15           184.22   
2024-01-08            181.91    182.085           181.99           182.15   
2024-01-09            181.18     183.92          182.085           181.99   
2024-01-10            185.56     184.35           183.92          182.085   

                      ...                           XOM  \
              Volume  ... Company Market Capitalization   
Date                  ...                                 
2024-01-04  71983570  ...                406312720000.0   
2024-01-05  62379661  ...                407543730000.0   
2024-01-08  59144470  ...                400753320000.0   
2024-01-09  42841809  ...                395789570000.0   
2024-01-10  46792908  ...                391897990000.0   

                                                                              \
           Company Market Capitalization_lag_1 Price Close Price Close_lag_1   
Date                                                                           
2024-01-04                      409886620000.0      102.32            103.22   
2024-01-05                      406312720000.0      102.63            102.32   
2024-01-08                      407543730000.0      100.92            102.63   
2024-01-09                      400753320000.0       99.67            100.92   
2024-01-10                      395789570000.0       98.69             99.67   

                                                                           \
           Price Close_lag_2 Price Open Price Open_lag_1 Price Open_lag_2   
Date                                                                        
2024-01-04            102.36     104.08           102.27           100.92   
2024-01-05            103.22     103.17           104.08           102.27   
2024-01-08            102.32     100.73           103.17           104.08   
2024-01-09            102.63     101.29           100.73           103.17   
2024-01-10            100.92       99.8           101.29           100.73   

                                   
              Volume Volume_lag_1  
Date                               
2024-01-04  19395154     23490779  
2024-01-05  15827392     19395154  
2024-01-08  23370127     15827392  
2024-01-09  19496647     23370127  
2024-01-10  18206088     19496647  

[5 rows x 275 columns]

In [188]:
y.head()

,AAPL.O,AMZN.O,AVGO.O,BAC,BRKb,COST.O,CRM,GOOG.O,GOOGL.O,HD,...,MSFT.O,NFLX.O,NVDA.O,ORCL.K,PG,TSLA.O,UNH,V,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2024-01-04,2.006487,3.13343,2.463275,1.065089,1.237352,2.057531,3.832988,1.803825,1.796319,2.758498,...,1.834538,2.182569,8.864953,2.017741,0.026909,1.059135,-1.63177,1.128616,0.310598,-1.368256
2024-01-05,2.185672,4.2206,3.165027,-2.323555,0.358325,1.702718,4.069767,3.76301,3.84587,0.947688,...,2.186268,1.693878,8.234719,0.876083,1.275268,-1.065308,0.184227,1.401671,1.659116,-2.884147
2024-01-08,0.339513,3.105298,0.532167,-1.639344,-0.070618,1.672989,1.249665,2.326905,2.477672,2.549363,...,2.156449,-1.381358,4.013167,-0.707051,0.840675,-2.707424,0.208753,0.769407,1.921011,-2.209671
2024-01-09,0.243059,2.517011,1.615719,-1.427297,-0.970292,0.959261,3.841739,0.77862,0.801703,2.986799,...,2.352378,2.103342,3.165224,1.100068,0.810449,-3.294178,0.243327,0.265826,1.129873,-1.003311
2024-01-10,-0.145013,0.578937,2.508861,-2.380952,-1.168732,1.557762,2.953091,0.305981,0.260051,-0.305493,...,1.489145,2.891309,0.662374,2.578907,0.440176,-6.433273,-3.000149,-0.147415,0.0186,1.276725


### Train, Test, Validation Split

Now we have our `X` and we need to predict `y` values. We will split the data into training, testing and validation sets.

In [189]:
from sklearn.model_selection import train_test_split
N = X.shape[0]
test_percent = 0.1
val_percent = 0.2

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_percent, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_percent/(1-test_percent), random_state=42)

print("Train size: ", X_train.shape[0])
print("Validation size: ", X_val.shape[0])
print("Test size: ", X_test.shape[0])

Train size:  129
Validation size:  37
Test size:  19


## Model

We first scale the features and then reduce the dimensionality of the data using PCA.

In [190]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

pipe = Pipeline([('scaler', StandardScaler()), ('pca', PCA(n_components=0.95))])
pipe.fit(X_train)

X_train_scaled = pipe.transform(X_train)

In [191]:
X_train_scaled[:5]

array([[-9.34607047,  5.84902894,  1.94973731,  1.95878848,  1.04652447,
         0.3052627 ,  0.06631669,  0.97345036,  0.9449907 ,  3.99954645,
         0.69458486, -2.57314861, -0.5859351 , -2.09174774,  2.47162967,
        -2.05715311,  2.47041936, -0.64943866, -0.29133012,  0.23167267,
         0.13525255,  0.28175193, -0.54603745, -0.45769433,  2.21906924,
         0.91057613, -0.63798894, -0.30136277, -0.91522462, -0.5602551 ,
        -1.5636851 , -0.03422442, -0.2854504 ],
       [-5.5798141 , -4.48397266,  7.51633204, -3.9566993 , -2.30296458,
        -1.81416469,  0.19846839, -4.50937725, -1.08449086, -5.12967452,
        -2.23254358, -2.45381456,  1.67387397,  2.61286408,  2.37187875,
        -2.12499534, -1.48527047,  0.05452435, -0.08134193, -0.06502289,
        -0.70022373, -1.61211219, -1.40111834, -1.3547879 , -1.29611077,
         1.41550806, -1.09564104,  1.48371742, -0.28266875, -1.13665995,
         0.1971621 , -0.04467984,  0.82157203],
       [ 4.3827727 , -6.8666

In [192]:
import torch
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [193]:
import torch.nn as nn
import torch.optim as optim

class StockPredictionNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(StockPredictionNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x
    
# Example parameters
input_size = X_train_scaled.shape[1]  # Number of features
hidden_size = 64  # Number of neurons in hidden layers
output_size = y_train.shape[1]  # Number of target variables

model = StockPredictionNN(input_size, hidden_size, output_size)
criterion = nn.MSELoss()  # Mean Squared Error Loss for regression
optimizer = optim.Adam(model.parameters(), lr=0.001)

model = model.to(device)

In [194]:
# Example training loop
num_epochs = 100
batch_size = 32

# Convert X_train and y_train to DataLoader for batching
X_train_tensor = torch.from_numpy(X_train_scaled).float()
y_train_tensor = torch.from_numpy(y_train.to_numpy(float)).float()
train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

for epoch in range(num_epochs):
    for inputs, targets in train_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [10/100], Loss: 2.2047
Epoch [20/100], Loss: 1.9924
Epoch [30/100], Loss: 8.7628
Epoch [40/100], Loss: 14.0034
Epoch [50/100], Loss: 6.7767
Epoch [60/100], Loss: 2.3492
Epoch [70/100], Loss: 3.7281
Epoch [80/100], Loss: 0.7574
Epoch [90/100], Loss: 1.4148
Epoch [100/100], Loss: 2.7419


In [195]:
torch.save(model.state_dict(), 'stock_prediction_model.pth')

In [196]:
# Score the model on training data
X_val_scaled = pipe.transform(X_val)
X_val_tensor = torch.from_numpy(X_val_scaled).float()
y_val_tensor = torch.from_numpy(y_val.to_numpy(float)).float()
y_val_tensor = y_val_tensor.to(device)
X_val_tensor = X_val_tensor.to(device)

model.eval()
with torch.no_grad():
    y_pred = model(X_val_tensor)
    val_loss = criterion(y_pred, y_val_tensor)
    print(f'Validation Loss: {val_loss.item():.4f}')
    


Validation Loss: 7.1225


In [197]:
# Testing
X_test_scaled = pipe.transform(X_test)
X_test_tensor = torch.from_numpy(X_test_scaled).float()
y_test_tensor = torch.from_numpy(y_test.to_numpy(float)).float()
y_test_tensor = y_test_tensor.to(device)
X_test_tensor = X_test_tensor.to(device)

model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor)
    test_loss = criterion(y_pred, y_test_tensor)
    print(f'Test Loss: {test_loss.item():.4f}')
    

Test Loss: 7.0147


##  Make Predictions with current data
We need to get data up to the largest lag we have used.

In [198]:
today = pd.to_datetime('2024-10-01')
max_lag = max([fields[field_name].lag for field_name in field_names])
print("Max lag: ", max_lag)
start_date = today - pd.DateOffset(days=max_lag+5) # Add 5 days to be safe across non-trading days
start_date = start_date.isoformat() + 'Z'
end_date = today.isoformat() + 'Z'



Max lag:  2


In [199]:
cur_data = ld.get_history(universe, list(fields.keys()), start=start_date, end=end_date, interval='1d')
cur_data.head()

c:\Users\tangy\Desktop\PortfolioManagement\qrtVenv\lib\site-packages\pandas\core\dtypes\cast.py:1056:RuntimeWarning: invalid value encountered in cast
c:\Users\tangy\Desktop\PortfolioManagement\qrtVenv\lib\site-packages\pandas\core\dtypes\cast.py:1080:RuntimeWarning: invalid value encountered in cast


AAPL.O                                          \
           1 Week Total Return    Volume Company Market Capitalization   
Date                                                                     
2024-09-24            4.880299  43556068                 3456964629690   
2024-09-25            2.573746  42308715                 3441760492690   
2024-09-26           -0.589855  36636707                 3459245250240   
2024-09-27           -0.179667  34025967                 3463350367230   
2024-09-30            2.883384  54793391                 3522211138000   

                                               AMZN.O            \
           Price Close Price Open 1 Week Total Return    Volume   
Date                                                              
2024-09-24      227.37    228.645            3.788527  43478926   
2024-09-25      226.37     224.93            3.272006  26391144   
2024-09-26      227.52      227.3            0.679412  36334854   
2024-09-27      227.79     228.46           -1.894572  36002316   
2024-09-30       233.0     230.04           -3.894161  41680400   

                                                                 ...  \
           Company Market Capitalization Price Close Price Open  ...   
Date                                                             ...   
2024-09-24           2035720152238.76001      193.96     194.27  ...   
2024-09-25          2020711491598.929932      192.53     193.75  ...   
2024-09-26          2006332564971.959961      191.16     194.31  ...   
2024-09-27          1972851706621.570068      187.97     190.68  ...   
2024-09-30               1958514630000.0      186.33     187.14  ...   

                           WMT                                          \
           1 Week Total Return    Volume Company Market Capitalization   
Date                                                                     
2024-09-24            2.633588  14651133           648445722206.579956   
2024-09-25            2.998861  14163785           654313645563.599976   
2024-09-26            2.409021  17061133           642417033826.079956   
2024-09-27            0.910701  11899050           641291678661.719971   
2024-09-30            0.522843  19008187                649088782300.5   

                                                  XOM            \
           Price Close Price Open 1 Week Total Return    Volume   
Date                                                              
2024-09-24       80.67      80.48            2.513575  11984863   
2024-09-25        81.4      80.89            0.165823  13816041   
2024-09-26       79.92      81.04           -2.758621  16887908   
2024-09-27       79.78       79.9            0.477141  15963973   
2024-09-30       80.75      79.89           -0.119291  13250657   

                                                                 
           Company Market Capitalization Price Close Price Open  
Date                                                             
2024-09-24                520032851189.0      117.05     117.84  
2024-09-25           509903206586.599976      114.77    116.485  
2024-09-26                501150838224.0       112.8     111.14  
2024-09-27           514568174495.599976      115.82     113.76  
2024-09-30           515192981509.919983      117.22     115.46  

[5 rows x 125 columns]

In [200]:
expanded_cur_data = expand_data(cur_data)
expanded_cur_data = expanded_cur_data.dropna()
expanded_cur_data.head()

AAPL.O                                \
           1 Week Total Return Company Market Capitalization   
Date                                                           
2024-09-26           -0.589855                 3459245250240   
2024-09-27           -0.179667                 3463350367230   
2024-09-30            2.883384                 3522211138000   
2024-10-01           -0.510182                 3419568161060   

                                                                              \
           Company Market Capitalization_lag_1 Price Close Price Close_lag_1   
Date                                                                           
2024-09-26                       3441760492690      227.52            226.37   
2024-09-27                       3459245250240      227.79            227.52   
2024-09-30                       3463350367230       233.0            227.79   
2024-10-01                       3522211138000      226.21             233.0   

                                                                           \
           Price Close_lag_2 Price Open Price Open_lag_1 Price Open_lag_2   
Date                                                                        
2024-09-26            227.37      227.3           224.93          228.645   
2024-09-27            226.37     228.46            227.3           224.93   
2024-09-30            227.52     230.04           228.46            227.3   
2024-10-01            227.79     229.52           230.04           228.46   

                      ...                           XOM  \
              Volume  ... Company Market Capitalization   
Date                  ...                                 
2024-09-26  36636707  ...                501150838224.0   
2024-09-27  34025967  ...           514568174495.599976   
2024-09-30  54793391  ...           515192981509.919983   
2024-10-01  63285048  ...            527103687702.47998   

                                                                              \
           Company Market Capitalization_lag_1 Price Close Price Close_lag_1   
Date                                                                           
2024-09-26                 509903206586.599976       112.8            114.77   
2024-09-27                      501150838224.0      115.82             112.8   
2024-09-30                 514568174495.599976      117.22            115.82   
2024-10-01                 515192981509.919983      119.93            117.22   

                                                                           \
           Price Close_lag_2 Price Open Price Open_lag_1 Price Open_lag_2   
Date                                                                        
2024-09-26            117.05     111.14          116.485           117.84   
2024-09-27            114.77     113.76           111.14          116.485   
2024-09-30             112.8     115.46           113.76           111.14   
2024-10-01            115.82     116.04           115.46           113.76   

                                   
              Volume Volume_lag_1  
Date                               
2024-09-26  16887908     13816041  
2024-09-27  15963973     16887908  
2024-09-30  13250657     15963973  
2024-10-01  23235878     13250657  

[4 rows x 275 columns]

In [ ]:
# predict based on the last row of the data
X_cur = expanded_cur_data.iloc[-1]
X_cur_scaled = pipe.transform(X_cur.values.reshape(1, -1))
X_cur_tensor = torch.from_numpy(X_cur_scaled).float()
X_cur_tensor = X_cur_tensor.to(device)
print(X_cur_tensor)

model.eval()
with torch.no_grad():
    y_pred = model(X_cur_tensor)

y_pred = y_pred.cpu().numpy()
y_pred

tensor([[ 1.5656e+01,  1.0963e+01, -1.9364e+00,  1.6349e+00, -2.2832e+00,
         -2.9140e+00,  2.9853e-01,  1.6038e-01, -1.3351e+00, -2.4007e+00,
         -5.5705e-01, -6.1374e-01, -6.7145e-01, -1.2483e+00,  6.5309e-01,
          9.7895e-01,  1.7751e+00, -5.0270e-01, -5.9707e-01,  2.3561e+00,
         -1.0109e-01,  9.0838e-01,  7.5454e-01, -1.7934e-02, -2.6906e+00,
         -8.7513e-01, -5.4624e-01,  4.8479e-01,  1.1635e+00, -5.8508e-01,
          8.2806e-03,  2.1811e-01,  5.7100e-01]], device='cuda:0')


array([[ 0.06145923, -0.15312374, -1.164156  , -0.71330297,  0.18035412,
        -0.23928311, -0.598305  , -0.243036  , -0.17071128,  0.86199504,
        -0.33928996, -0.8288686 , -0.2928587 , -0.11921224,  0.792503  ,
        -0.36453104,  0.7811218 , -3.3112037 ,  0.29967964, -0.19500342,
         1.802981  ,  0.06105877, -0.3503624 ,  0.8093217 ,  1.7062104 ]],
      dtype=float32)